In [6]:
import torch # torch 파일 불러오는 용도
import torch.nn as nn
import torch.onnx # torch, onnx 변환 용도
import onnx # onnx 불러오고 실행
import onnxruntime
import numpy as np

In [2]:
# 신경망 함수들 정의

## Encoder
class Encoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.LSTM(13, 13, batch_first = True)
    def forward(self, x) :
        y, hc = self.rnn(x)
        return y, hc
        
## Decoder 
class Decoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Linear(13, 7)
    def forward(self, hc) :
        h = hc[0] # h, c 중에서 h만 골라내기
        h = h.reshape(-1, 13) # 3차원을 2차원으로 바꾸기
        y = self.f(h)
        return y

In [3]:
## Encoder
# 함수 불러오기

encoder = torch.load('encoder.pt', weights_only=False)
encoder.eval()
print(encoder)

Encoder(
  (rnn): LSTM(13, 13, batch_first=True)
)


In [4]:
# onnx로 변환
x = torch.randn(1,100,13) #더미 입력값
dynamic_axes = {'input' : {0 : 'b', 1 : 'f'}, 'output' : {0 : 'b', 1 : 'f'}, 'output_h' : {1 : 'b'}, 'output_c' : {1 : 'b'}}

torch.onnx.export(
    encoder, # 변환할 신경망 함수
    x, #신경망에 입력값
    "encoder.onnx", #저장 파일 이름
    export_params=True, #w,b값 저장
    opset_version=10, #onnx 버전
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output', 'output_h', 'output_c'],
    dynamic_axes=dynamic_axes #입력값과 출력값의 가변 차원 지정
)


C:\Users\Administrator\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [5]:
# 잘 저장되었는지 테스트

onnx_encoder = onnx.load('encoder.onnx')
onnx.checker.check_model(onnx_encoder)

In [9]:
# torch와 onnx 결과값 비교
x = torch.randn(2,50,13)

onnx_encoder = onnxruntime.InferenceSession('encoder.onnx', providers=['CPUExecutionProvider'])

numpy_x = x.numpy().astype(np.float32)

inputs = {onnx_encoder.get_inputs()[0].name : numpy_x}
outputs = onnx_encoder.run(None, inputs)
y, hc = encoder(x)

np.testing.assert_allclose(y.detach().numpy(), outputs[0], rtol=1e-03, atol=1e-05) #"LSTM 일반 출력값"
np.testing.assert_allclose(hc[0].detach().numpy(), outputs[1], rtol=1e-03, atol=1e-05) #"h값"
np.testing.assert_allclose(hc[1].detach().numpy(), outputs[2], rtol=1e-03, atol=1e-05) #"c값"

In [11]:
##decoder
# decoder 불러오기

decoder = torch.load('decoder.pt', weights_only=False)
decoder.eval()

print(decoder)

Decoder(
  (f): Linear(in_features=13, out_features=7, bias=True)
)


In [18]:
# onnx 변환
h = torch.randn(1,1,13) 
c = torch.randn(1,1,13)
dynamic_axes = {'input_h' : {1 : 'b'}, 'input_c' : {1 : 'b'}, 'output' : {0 : 'b'}}

torch.onnx.export(
    decoder,
    ((h, c),),
    'decoder.onnx',
    export_params = True,
    opset_version = 10,
    do_constant_folding= True,
    input_names= ['input_h', 'input_c'],
    output_names= ['output'],
    dynamic_axes= dynamic_axes
)

In [19]:
#변환 잘 되었는지 테스트

onnx_decoder = onnx.load('decoder.onnx')
onnx.checker.check_model(onnx_decoder)

In [24]:
numpy_h = h.numpy()
numpy_c = c.numpy()

onnx_decoder = onnxruntime.InferenceSession('decoder.onnx', providers=['CPUExecutionProvider'])

inputs = {onnx_decoder.get_inputs()[0].name : numpy_h}
outputs = onnx_decoder.run(None, inputs)

y = decoder((h, c))

np.testing.assert_allclose(y.detach().numpy(), outputs[0], rtol=1e-03, atol=1e-05) #"LSTM 일반 출력값"

print(y)
print(outputs[0])

tensor([[-1.1032, -1.3631, -1.6473,  3.5648, -0.6208,  0.3505,  4.1633]],
       grad_fn=<AddmmBackward0>)
[[-1.1032197  -1.3630816  -1.6472695   3.5647705  -0.62077326  0.35049063
   4.1633234 ]]
